In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [28]:
bus = pd.read_csv("/Users/Jenny/Desktop/seoulbigcam/Raw Data/서울시 대중교통시설 위치정보_bus.csv", encoding="CP949")
subway = pd.read_csv("/Users/Jenny/Desktop/seoulbigcam/Raw Data/서울시 대중교통시설 위치정보_sub.csv", encoding="CP949")

### 지하철 데이터 전처리

In [29]:
subway.head()

,`GU_NM`,`GU_CD`,`SUB_STA_SN`,`KOR_SUB_NM`,`POINT_X`,`POINT_Y`
0,`중랑구`,`11260`,`40`,`용마산역`,`207607`,`452624`
1,`강북구`,`11305`,`41`,`미아삼거리역`,`202663`,`457082`
2,`강북구`,`11305`,`42`,`수유역`,`202238`,`459793`
3,`강북구`,`11305`,`43`,`미아역`,`202304`,`458542`
4,`강동구`,`11740`,`44`,`상일동역`,`214745`,`450818`


In [30]:
column_name = ['`GU_NM`', '`GU_CD`', '`SUB_STA_SN`', '`KOR_SUB_NM`', '`POINT_X`', '`POINT_Y`']

for i in column_name:
    subway[i] = subway[i].str[1:-1]

subway.head()

,`GU_NM`,`GU_CD`,`SUB_STA_SN`,`KOR_SUB_NM`,`POINT_X`,`POINT_Y`
0,중랑구,11260,40,용마산역,207607,452624
1,강북구,11305,41,미아삼거리역,202663,457082
2,강북구,11305,42,수유역,202238,459793
3,강북구,11305,43,미아역,202304,458542
4,강동구,11740,44,상일동역,214745,450818


In [31]:
subway = subway[['`GU_NM`', '`KOR_SUB_NM`', '`POINT_X`', '`POINT_Y`']]

In [32]:
subway = subway.rename(columns={'`GU_NM`': 'gu',
                               '`KOR_SUB_NM`': 'sub_name',
                               '`POINT_X`':'X_VALUE',
                               '`POINT_Y`':'Y_VALUE'})
subway.head()


,gu,sub_name,X_VALUE,Y_VALUE
0,중랑구,용마산역,207607,452624
1,강북구,미아삼거리역,202663,457082
2,강북구,수유역,202238,459793
3,강북구,미아역,202304,458542
4,강동구,상일동역,214745,450818


In [33]:
# X,Y좌표 경도 위도로 변환하기

sub = gpd.GeoDataFrame(subway, geometry=gpd.points_from_xy(subway.X_VALUE, subway.Y_VALUE))

sub.crs = 'epsg:5181'
sub = sub.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
sub['x'] = sub['geometry'].x
sub['y'] = sub['geometry'].y

# 필요한 컬럼만 추출해서 저장하기
sub = sub[['sub_name', 'x', 'y']]

print(sub)

    sub_name           x          y
0       용마산역  127.086111  37.573129
1     미아삼거리역  127.030161  37.613322
2        수유역  127.025356  37.637749
3        미아역  127.026100  37.626478
4       상일동역  127.166877  37.556770
..       ...         ...        ...
281      뚝섬역  127.047358  37.547184
282      성수역  127.056081  37.544504
283      옥수역  127.018466  37.541101
284     도봉산역  127.046439  37.689648
285      방학역  127.044340  37.667386

[286 rows x 3 columns]


In [12]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [37]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in sub.iterrows():
    get_location(row)
    
    # 20개씩 처리될 때 메시지 출력
    if i % 20 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
sub['city'] = [location.get('city') for location in locations]
sub['gu'] = [location.get('gu') for location in locations]
sub['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
20 번째 경위도 좌표계에 대한 정보 불러오는 중...
40 번째 경위도 좌표계에 대한 정보 불러오는 중...
60 번째 경위도 좌표계에 대한 정보 불러오는 중...
80 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
120 번째 경위도 좌표계에 대한 정보 불러오는 중...
140 번째 경위도 좌표계에 대한 정보 불러오는 중...
160 번째 경위도 좌표계에 대한 정보 불러오는 중...
180 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
220 번째 경위도 좌표계에 대한 정보 불러오는 중...
240 번째 경위도 좌표계에 대한 정보 불러오는 중...
260 번째 경위도 좌표계에 대한 정보 불러오는 중...
280 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [46]:
# 지하철 데이터 저장
sub.to_csv('/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 지하철역 위치 정보.csv')

### 버스 데이터 전처리

In [8]:
bus.columns

Index(['`YYYY_MM`', '`LINE_NO_EXP`', '`LINE_NO`', '`LINE_EXP`', '`SEQ_NO`',
       '`BUS_STA_NM`', '`X_COORD`', '`Y_COORD`', '`ARSID`'],
      dtype='object')

In [42]:
column_name2 = ['`YYYY_MM`', '`LINE_NO_EXP`', '`LINE_NO`', '`LINE_EXP`', '`SEQ_NO`',
       '`BUS_STA_NM`', '`X_COORD`', '`Y_COORD`', '`ARSID`']

for i in column_name2:
    bus[i] = bus[i].str[1:-1]
bus.head()

,`YYYY_MM`,`LINE_NO_EXP`,`LINE_NO`,`LINE_EXP`,`SEQ_NO`,`BUS_STA_NM`,`X_COORD`,`Y_COORD`,`ARSID`
0,201810,105번(상계동~서울역),105번,상계동~서울역,85,상계주공7단지.광림교회앞,127.06278,37.6605,11188
1,201810,1129번(상계현대2차아파트~창동역동측),1129번,상계현대2차아파트~창동역동측,6,상계주공7단지.광림교회앞,127.06278,37.6605,11188
2,201810,146번(상계주공7단지~강남역),146번,상계주공7단지~강남역,133,상계주공7단지.광림교회앞,127.06278,37.6605,11188
3,201810,N13번(상계동차고지~송파공영차고지),N13번,상계동차고지~송파공영차고지,147,상계주공7단지.광림교회앞,127.06278,37.6605,11188
4,201810,N13번(송파공영차고지~상계동차고지),N13번,송파공영차고지~상계동차고지,71,상계주공7단지.광림교회앞,127.06278,37.6605,11188


In [43]:
bus = bus[['`BUS_STA_NM`', '`X_COORD`',	'`Y_COORD`']]
bus = bus.rename(columns={'`BUS_STA_NM`': 'name',
                        '`X_COORD`': 'x',
                        '`Y_COORD`': 'y'})
bus

,name,x,y
0,상계주공7단지.광림교회앞,127.06278,37.6605
1,상계주공7단지.광림교회앞,127.06278,37.6605
2,상계주공7단지.광림교회앞,127.06278,37.6605
3,상계주공7단지.광림교회앞,127.06278,37.6605
4,상계주공7단지.광림교회앞,127.06278,37.6605
...,...,...,...
38418,서울중앙외과,127.09436,37.58781
38419,농협앞,127.09181,37.58941
38420,면목역,127.08697,37.58831
38421,KT중랑지점,127.07818,37.59076


In [45]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in bus.iterrows():
    get_location(row)
    
    # 300개씩 처리될 때 메시지 출력
    if i % 300 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
bus['city'] = [location.get('city') for location in locations]
bus['gu'] = [location.get('gu') for location in locations]
bus['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1500 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2100 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2700 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보 불러오는 중...
3300 번째 경위도 좌표계에 대한 정보 불러오는 중...
3600 번째 경위도 좌표계에 대한 정보 불러오는 중...
3900 번째 경위도 좌표계에 대한 정보 불러오는 중...
4200 번째 경위도 좌표계에 대한 정보 불러오는 중...
4500 번째 경위도 좌표계에 대한 정보 불러오는 중...
4800 번째 경위도 좌표계에 대한 정보 불러오는 중...
5100 번째 경위도 좌표계에 대한 정보 불러오는 중...
5400 번째 경위도 좌표계에 대한 정보 불러오는 중...
5700 번째 경위도 좌표계에 대한 정보 불러오는 중...
6000 번째 경위도 좌표계에 대한 정보 불러오는 중...
6300 번째 경위도 좌표계에 대한 정보 불러오는 중...
6600 번째 경위도 좌표계에 대한 정보 불러오는 중...
6900 번째 경위도 좌표계에 대한 정보 불러오는 중...
7200 번째 경위도 좌표계에 대한 정보 불러오는 중...
7500 번째 경위도 좌표계에 대한 정보 불러오는 중...
7800 번째 경위도 좌표계에 대한 정보 불러오는 중...
8100 번째 경위도 좌표계에 대한 정보 불러오는 중...
8400 번째 경위도 좌표계에 대한 정보 불러오는 중...
8700 번째 경위도 좌표계에 대한 정보 불러오는 중...
9000 번째 경위도 좌표계에

In [62]:
bus = bus[bus['city'] == '서울특별시']
bus.to_csv('/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 버스 정류장 위치 정보.csv')

### 면적 당 지하철역/버스정류장 산출

In [63]:
bus_no_dup = bus.drop_duplicates()
sub_no_dup = sub.drop_duplicates()

In [64]:
sub_new = sub_no_dup.groupby(['city', 'gu', 'dong'], as_index=False)['sub_name'].count()
sub_new

,city,gu,dong,sub_name
0,서울특별시,강남구,개포2동,3
1,서울특별시,강남구,논현1동,3
2,서울특별시,강남구,논현2동,2
3,서울특별시,강남구,대치1동,1
4,서울특별시,강남구,대치2동,2
...,...,...,...,...
203,서울특별시,중랑구,묵1동,1
204,서울특별시,중랑구,상봉1동,1
205,서울특별시,중랑구,상봉2동,1
206,서울특별시,중랑구,신내1동,1


In [65]:
bus_new = bus_no_dup.groupby(['city', 'gu', 'dong'], as_index=False)['name'].count()
bus_new

,city,gu,dong,name
0,서울특별시,강남구,개포1동,18
1,서울특별시,강남구,개포2동,38
2,서울특별시,강남구,개포3동,10
3,서울특별시,강남구,개포4동,31
4,서울특별시,강남구,논현1동,19
...,...,...,...,...
421,서울특별시,중랑구,상봉2동,15
422,서울특별시,중랑구,신내1동,47
423,서울특별시,중랑구,신내2동,21
424,서울특별시,중랑구,중화1동,14


In [66]:
# 행정동별 면적 데이터 로드

area = pd.read_csv("/Users/Jenny/Desktop/seoulbigcam/Raw Data/행정동별 면적.csv")

area = area.drop(area.index[0:3])     # 필요없는 데이터 정리
area = area.reset_index(drop=True)    # 인덱스 리셋
area = area.drop('동별(1)', axis=1)    # 필요없는 컬럼 삭제
area = area.rename(columns={'동별(2)': 'gu',
                            '동별(3)':'dong',
                            '2022':'area_km2'})     # 컬럼명 변경

area.head()

,gu,dong,area_km2
0,종로구,소계,23.91
1,종로구,사직동,1.23
2,종로구,삼청동,1.49
3,종로구,부암동,2.27
4,종로구,평창동,8.87


In [74]:
sub_merge = sub_new.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
sub_merge['area_km2'] = sub_merge['area_km2'].astype(float)
sub_merge['density_sub'] = sub_merge['sub_name'] / sub_merge['area_km2']

,city,gu,dong,sub_name,area_km2,density_sub
0,서울특별시,강남구,개포2동,3,2.51,1.195219
1,서울특별시,강남구,논현1동,3,1.25,2.400000
2,서울특별시,강남구,논현2동,2,1.47,1.360544
3,서울특별시,강남구,대치1동,1,0.79,1.265823
4,서울특별시,강남구,대치2동,2,2.00,1.000000
...,...,...,...,...,...,...
203,서울특별시,중랑구,묵1동,1,1.20,0.833333
204,서울특별시,중랑구,상봉1동,1,0.88,1.136364
205,서울특별시,중랑구,상봉2동,1,0.65,1.538462
206,서울특별시,중랑구,신내1동,1,2.56,0.390625


In [79]:
bus_merge = bus_new.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
bus_merge['area_km2'] = bus_merge['area_km2'].astype(float)
bus_merge['density_sub'] = bus_merge['name'] / bus_merge['area_km2']
bus_merge

,city,gu,dong,name,area_km2,density_sub
0,서울특별시,강남구,개포1동,18,1.27,14.173228
1,서울특별시,강남구,개포2동,38,2.51,15.139442
2,서울특별시,강남구,개포3동,10,1.24,8.064516
3,서울특별시,강남구,개포4동,31,1.49,20.805369
4,서울특별시,강남구,논현1동,19,1.25,15.200000
...,...,...,...,...,...,...
421,서울특별시,중랑구,상봉2동,15,0.65,23.076923
422,서울특별시,중랑구,신내1동,47,2.56,18.359375
423,서울특별시,중랑구,신내2동,21,0.93,22.580645
424,서울특별시,중랑구,중화1동,14,0.64,21.875000


In [80]:
bus_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 0 to 425
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         426 non-null    object 
 1   gu           426 non-null    object 
 2   dong         426 non-null    object 
 3   name         426 non-null    int64  
 4   area_km2     426 non-null    float64
 5   density_sub  426 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 23.3+ KB


In [81]:
sub_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 0 to 207
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         208 non-null    object 
 1   gu           208 non-null    object 
 2   dong         208 non-null    object 
 3   sub_name     208 non-null    int64  
 4   area_km2     208 non-null    float64
 5   density_sub  208 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 19.5+ KB
